Documentacion: https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/

In [3]:
from cassandra.cluster import Cluster

#Conexion al cluster
#Es importante haber mapeado el puerto usando la bandera -p 9042:9042 al crear el contenedor

cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect('bdnosql') #El keyspace lo creé desde la terminal

In [5]:
# Insertar cosas a la tabla estaciones (la tabla la creé desde la terminal)

sentencia = session.prepare( 
    """
    INSERT INTO estaciones (id, nombre)
    VALUES (?, ?)
    """
)

row = (2, 'Verano')
session.execute(sentencia, row)

In [7]:
row = (3, 'Otoño')
resultado = session.execute(sentencia, row)

print(resultado)

In [9]:
# Leer cosas de la tabla
rows = session.execute('SELECT id, nombre FROM estaciones')
for row in rows:
    print(row.id, row.nombre)

1 Primavera
2 Verano
3 Otoño
